In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

#### Sanity Check

In [2]:
df = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
for perc, col in zip(df.isnull().sum().index, df.isnull().sum()):
    if(col != 0.0):
        print(f'{perc} --> {round(col*100/len(df), 4)}% nulos')

LotFrontage --> 17.7397% nulos
Alley --> 93.7671% nulos
MasVnrType --> 0.5479% nulos
MasVnrArea --> 0.5479% nulos
BsmtQual --> 2.5342% nulos
BsmtCond --> 2.5342% nulos
BsmtExposure --> 2.6027% nulos
BsmtFinType1 --> 2.5342% nulos
BsmtFinType2 --> 2.6027% nulos
Electrical --> 0.0685% nulos
FireplaceQu --> 47.2603% nulos
GarageType --> 5.5479% nulos
GarageYrBlt --> 5.5479% nulos
GarageFinish --> 5.5479% nulos
GarageQual --> 5.5479% nulos
GarageCond --> 5.5479% nulos
PoolQC --> 99.5205% nulos
Fence --> 80.7534% nulos
MiscFeature --> 96.3014% nulos


In [5]:
#LotFrontage é o único com NaNs em potencial, pois os demais representam alguma informação, como quantia 0.
#incialmente essa coluna será dropada. Outra opção seria estimar o dado através de casas com caracteristicas próximas

In [6]:
df_copy = df.drop(['LotFrontage'], axis=1).copy()
df_copy.dropna(subset=['Electrical'], inplace=True)

In [7]:
PoolQC_to_numeric = {'PoolQC': {None:0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4}}
Fence_to_numeric = {'Fence': {None:-1, 'MnWw':0, 'GdWo':1, 'MnPrv':0, 'GdPrv':1}}
GarageCond_to_numeric = {'GarageCond': {None:0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}}
GarageQual_to_numeric = {'GarageQual': {None:0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}}
GarageFinish_to_numeric = { 'GarageFinish' : {None:0, 'Unf':1, 'RFn':2, 'Fin':3}}
FireplaceQu_to_numeric = {'FireplaceQu': {None:0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}}
BsmtFinType2_to_numeric = {'BsmtFinType2': {None:0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}}
BsmtFinType1_to_numeric = {'BsmtFinType1': {None:0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}}
BsmtExposure_to_numeric = {'BsmtExposure': {None:0, 'No':0, 'Mn':1, 'Av':2, 'Gd':3}}
BsmtCond_to_numeric = {'BsmtCond': {None:0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}}
BsmtQual_to_numeric = {'BsmtQual': {None:0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}}

In [8]:
df_copy['MiscFeature'] = df['MiscFeature'].map(lambda x: 1 if (x=='Elev' or x=='Gar2' or x=='Othr' or x=='Shed' or x=='TenC') else 0)
df_copy['GarageType'] = df['GarageType'].map(lambda x: 1 if (x=='2Types' or x=='Attchd' or x=='Basment' or x=='BuiltIn' or x=='CarPort') else 0)
df_copy['GarageYrBlt'] = df['GarageYrBlt'].map(lambda x: 1 if x>0 else 0)
df_copy['MasVnrArea'] = df['MasVnrArea'].map(lambda x: 1 if x>0 else 0)
df_copy['Alley'] = df_copy['Alley'].map(lambda x: 2 if x=='Pave' else (1 if x=='Grvl' else 0))
df_copy['MasVnrType'] = df['MasVnrType'].map(lambda x: 1 if (x=='BrkCmn' or x=='BrkFace' or x=='CBlock' or x=='Stone') else 0)
df_copy.replace(PoolQC_to_numeric, inplace=True)
df_copy.replace(Fence_to_numeric, inplace=True)
df_copy.replace(GarageCond_to_numeric, inplace=True)
df_copy.replace(GarageQual_to_numeric, inplace=True)
df_copy.replace(GarageFinish_to_numeric, inplace=True)
df_copy.replace(FireplaceQu_to_numeric, inplace=True)
df_copy.replace(BsmtFinType2_to_numeric, inplace=True)
df_copy.replace(BsmtFinType1_to_numeric, inplace=True)
df_copy.replace(BsmtExposure_to_numeric, inplace=True)
df_copy.replace(BsmtCond_to_numeric, inplace=True)
df_copy.replace(BsmtQual_to_numeric, inplace=True)

In [9]:
for perc, col in zip(df_copy.isnull().sum().index, df_copy.isnull().sum()):
    if(col != 0.0):
        print(f'{perc} --> {round(col*100/len(df_copy), 4)}% nulos')

In [10]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df_copy)
categorical_columns

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [11]:
df_categorical = df_copy[categorical_columns]
df_categorical.head(3)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Foundation,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,PConc,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,CBlock,GasA,Ex,Y,SBrkr,TA,Typ,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,PConc,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal


In [12]:
numeric_columns = [col for col in df_copy.columns if col not in categorical_columns]
df_numeric = df_copy[numeric_columns]
df_numeric.head(3)

,Id,MSSubClass,LotArea,Alley,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrType,MasVnrArea,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SalePrice
0,1,60,8450,0,7,5,2003,2003,1,1,...,0,0,0,0,-1,0,0,2,2008,208500
1,2,20,9600,0,6,8,1976,1976,0,0,...,0,0,0,0,-1,0,0,5,2007,181500
2,3,60,11250,0,7,5,2001,2002,1,1,...,0,0,0,0,-1,0,0,9,2008,223500


In [13]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
encoded = encoder.fit_transform(df_categorical)
encoded

array([[3., 1., 3., ..., 2., 8., 4.],
       [3., 1., 3., ..., 2., 8., 4.],
       [3., 1., 0., ..., 2., 8., 4.],
       ...,
       [3., 1., 3., ..., 2., 8., 4.],
       [3., 1., 3., ..., 2., 8., 4.],
       [3., 1., 3., ..., 2., 8., 4.]])

In [14]:
df_categorical_encoded = pd.DataFrame(data=encoded, columns=df_categorical.columns)
df_categorical_encoded.head(3)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Foundation,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,3.0,1.0,3.0,3.0,0.0,4.0,0.0,5.0,2.0,2.0,...,2.0,1.0,0.0,1.0,4.0,2.0,6.0,2.0,8.0,4.0
1,3.0,1.0,3.0,3.0,0.0,2.0,0.0,24.0,1.0,2.0,...,1.0,1.0,0.0,1.0,4.0,3.0,6.0,2.0,8.0,4.0
2,3.0,1.0,0.0,3.0,0.0,4.0,0.0,5.0,2.0,2.0,...,2.0,1.0,0.0,1.0,4.0,2.0,6.0,2.0,8.0,4.0


#### Machine Learning

In [15]:
from sklearn.model_selection import train_test_split

df_model = pd.concat([df_numeric, df_categorical_encoded], axis=1)
df_model.dropna(inplace=True)
target = df_model['SalePrice']
df_model.drop(['SalePrice'], inplace=False, axis=1)

x_train, x_teste, y_train, y_teste = train_test_split(df_model, target)

In [18]:
from sklearn import svm

clf = svm.SVR()

clf.fit(x_train, y_train)

SVR()

In [19]:
pred = clf.predict(x_teste)

In [21]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_teste, pred)

5063345638.728785

In [22]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, df_model, target, cv=5)
scores

array([-0.06704768, -0.05791583, -0.05362446, -0.01316164, -0.05375519])

In [23]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

-0.05 accuracy with a standard deviation of 0.02


#### Evaluation

In [24]:
df_test = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [78]:
df_copy = df_test.drop(['LotFrontage'], axis=1).copy()

df_copy['MiscFeature'] = df_test['MiscFeature'].map(lambda x: 1 if (x=='Elev' or x=='Gar2' or x=='Othr' or x=='Shed' or x=='TenC') else 0)
df_copy['GarageType'] = df_test['GarageType'].map(lambda x: 1 if (x=='2Types' or x=='Attchd' or x=='Basment' or x=='BuiltIn' or x=='CarPort') else 0)
df_copy['GarageYrBlt'] = df_test['GarageYrBlt'].map(lambda x: 1 if x>0 else 0)
df_copy['MasVnrArea'] = df_test['MasVnrArea'].map(lambda x: 1 if x>0 else 0)
df_copy['Alley'] = df_copy['Alley'].map(lambda x: 2 if x=='Pave' else (1 if x=='Grvl' else 0))
df_copy['MasVnrType'] = df_test['MasVnrType'].map(lambda x: 1 if (x=='BrkCmn' or x=='BrkFace' or x=='CBlock' or x=='Stone') else 0)
df_copy.replace(PoolQC_to_numeric, inplace=True)
df_copy.replace(Fence_to_numeric, inplace=True)
df_copy.replace(GarageCond_to_numeric, inplace=True)
df_copy.replace(GarageQual_to_numeric, inplace=True)
df_copy.replace(GarageFinish_to_numeric, inplace=True)
df_copy.replace(FireplaceQu_to_numeric, inplace=True)
df_copy.replace(BsmtFinType2_to_numeric, inplace=True)
df_copy.replace(BsmtFinType1_to_numeric, inplace=True)
df_copy.replace(BsmtExposure_to_numeric, inplace=True)
df_copy.replace(BsmtCond_to_numeric, inplace=True)
df_copy.replace(BsmtQual_to_numeric, inplace=True)

In [79]:
df_ids = pd.read_csv('house-prices-advanced-regression-techniques/sample_submission.csv')

In [83]:
categorical_columns = categorical_columns_selector(df_copy)
df_categorical = df_copy[categorical_columns]
numeric_columns = [col for col in df_copy.columns if col not in categorical_columns]
df_numeric = df_copy[numeric_columns]

encoded = encoder.fit_transform(df_categorical)
df_categorical_encoded = pd.DataFrame(data=encoded, columns=df_categorical.columns)

df_validation = pd.concat([df_numeric, df_categorical_encoded], axis=1)
df_validation = pd.concat([df_ids['Id'], df_validation['SalePrice']], axis=1)

df_validation.fillna(0, inplace=True)
df_validation.drop(1459, inplace=True)

In [85]:
df_validation['Id'] = df_validation['Id'].astype(int)
df_validation.to_csv('house-prices-advanced-regression-techniques/subs/sub01.csv', index=False)